In [6]:
import pandas as pd
import folium
from pyproj import Transformer

# Load data
df = pd.read_csv("/content/drive/MyDrive/APPLICATION/TfL_Traffic_Data/River_Piers.csv")

# Coordinate transformation (OSGB36 to WGS84)
transformer = Transformer.from_crs("EPSG:27700", "EPSG:4326")
lat, lon = transformer.transform(df["X"], df["Y"])
df["Latitude"] = lat
df["Longitude"] = lon

# Save cleaned data
df.to_csv("/content/drive/MyDrive/APPLICATION/TfL_Traffic_Data/Cleaned_River_Piers.csv", index=False)

In [7]:
df

,X,Y,NAME,OBJECTID,STATUS,Latitude,Longitude
0,530126.0000,178041.0000,St. George Wharf Pier,1,Current,51.486357,-0.127261
1,531499.0000,180787.0000,Blackfriars Pier,2,Current,51.510717,-0.106476
2,543845.0000,179473.0000,Woolwich Royal Arsenal Pier,3,Current,51.495904,0.070777
3,539551.0000,180056.0000,North Greenwich Pier,4,Current,51.502219,0.009191
4,538365.0000,178017.0000,Greenwich Pier,5,Current,51.484187,-0.008684
5,537421.0000,178343.0000,Masthouse Terrace Pier,6,Current,51.487347,-0.022144
6,536729.0000,179154.0000,Greenland Pier,7,Current,51.494802,-0.031790
7,533426.0625,180461.9063,Tower Pier,8,Current,51.507344,-0.078846
8,533035.0000,180404.0000,London Bridge City Pier,9,Current,51.506916,-0.084499
9,530546.0000,179949.0000,London Eye Pier,10,Current,51.503407,-0.120511


In [8]:
# Create a Folium map centered on London
tfl_map = folium.Map(location=[51.5074, -0.1278], zoom_start=12)

# Add markers for each pier
for idx, row in df.iterrows():
    color = "blue" if row["STATUS"] == "Current" else "red"
    popup = folium.Popup(
        f"<b>{row['NAME']}</b><br>Status: {row['STATUS']}",
        max_width=250
    )
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=popup,
        icon=folium.Icon(color=color)
    ).add_to(tfl_map)

# Save the map
tfl_map.save("/content/drive/MyDrive/APPLICATION/TfL_Traffic_Data/TfL_River_Piers_Map.html")

In [9]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Calculate nearest neighbor distances for current piers
current_piers = df[df["STATUS"] == "Current"]
coords = current_piers[["Latitude", "Longitude"]].values

nbrs = NearestNeighbors(n_neighbors=2).fit(coords)
distances, _ = nbrs.kneighbors(coords)
avg_distance = np.mean(distances[:, 1]) * 111  # Convert degrees to km (~111 km/degree)
print(f"Average distance between piers: {avg_distance:.2f} km")

# Identify gaps (areas with >2km between piers)
gaps = current_piers[distances[:, 1] * 111 > 2]
print("Potential gaps at:", gaps["NAME"].tolist())

Average distance between piers: 1.10 km
Potential gaps at: ['North Greenwich Pier', 'Hampton Court Pier', 'Kew Pier']


In [12]:
# Pseudo-code for network analysis (use NetworkX or OSMnx)
import networkx as nx

# Create a graph of piers
G = nx.Graph()
for _, row in df.iterrows():
    G.add_node(row["NAME"], pos=(row["Latitude"], row["Longitude"]))

# Add edges (hypothetical travel times)
for i in range(len(df)):
    for j in range(i+1, len(df)):
        node1 = df.iloc[i]["NAME"]
        node2 = df.iloc[j]["NAME"]
        distance = np.hypot(
            df.iloc[i]["Latitude"] - df.iloc[j]["Latitude"],
            df.iloc[i]["Longitude"] - df.iloc[j]["Longitude"]
        )
        G.add_edge(node1, node2, weight=distance)

# Find shortest path (example)
shortest_path = nx.shortest_path(G, source="Greenland Pier", target="Canary Wharf Pier", weight="weight")
print("Shortest path:", shortest_path)

Shortest path: ['Greenland Pier', 'Canary Wharf Pier']
